# Simple DMRG (Hubbard Hamiltonian)

## Setup Julia and DMRG environment

In [1]:
using Revise
includet("../src/dmrg.jl")

-0.5440211108893698

# Hubbard Hamiltonian 10 sites

## SU2 DMRG

In [2]:
Nsites=10
Nelec=10
Ssites=0.0
max_δnel = 2
max_δs = 1.5
@time begin
    schedule = [[16,1],[32,1]]
end

# By default t=1 and U=1


  0.000002 seconds (3 allocations: 288 bytes)


2-element Vector{Vector{Int64}}:
 [16, 1]
 [32, 1]

## Do the calculation

In [3]:
@time begin
    dmrg2site_1D_withSchedule(Nsites,Ssites,Nelec,max_δnel,max_δs,schedule)
end

||         DMRG  SCHEDULE BEGIN               ||
||         DMRG  SCHEDULE  1 :> M =    16      ||
Normalize=1.0
Starting DMRG Sweeps from: 5

||         Nsites     = 10                   ||
||         DMRG  Sweep=  1                   ||
||         Threshold  = 0.01000              ||
||         M          = 16                ||
|           Energy = -10.000000000000000       |



SweepR... 100%|██████████████████████████████████████████| Time: 0:00:14
SweepL... 100%|██████████████████████████████████████████| Time: 0:00:03



||         Nsites     = 10                   ||
||         DMRG  END                          ||
||         Threshold  = 0.01000              ||
||         M          = 16                ||
|           Energy = -9.763671194925331       |
|           ΔE     = -0.236328805074669       |

 ────────────────────────────────────────────
                               Time          
                      ───────────────────────
   Tot / % measured:        126s /  13.9%    

 Section      ncalls     time    %tot     avg
 ────────────────────────────────────────────
 diag Heff        12    8.32s   47.6%   693ms
 calc Heff         2    4.66s   26.7%   2.33s
 move Gauge       12    3.35s   19.2%   279ms
 move Heff        10    1.13s    6.5%   113ms
 ────────────────────────────────────────────||         DMRG  SCHEDULE  2 :> M =    32      ||
Starting DMRG Sweeps from: 5

||         Nsites     = 10                   ||
||         DMRG  Sweep=  1                   ||
||         Threshold  = 0.0100

SweepR... 100%|██████████████████████████████████████████| Time: 0:00:06
SweepL... 100%|██████████████████████████████████████████| Time: 0:00:01



||         Nsites     = 10                   ||
||         DMRG  END                          ||
||         Threshold  = 0.01000              ||
||         M          = 32                ||
|           Energy = -9.766216827240608       |
|           ΔE     = 0.002545632315277       |

 ────────────────────────────────────────────
                               Time          
                      ───────────────────────
   Tot / % measured:        135s /  18.9%    

 Section      ncalls     time    %tot     avg
 ────────────────────────────────────────────
 diag Heff        24    14.8s   57.8%   616ms
 calc Heff         4    5.30s   20.7%   1.33s
 move Gauge       24    3.42s   13.4%   143ms
 move Heff        20    2.06s    8.1%   103ms
 ────────────────────────────────────────────U=(3, 3, 3) D=(1,) Vt=(1,)
U=(1,) D=(3, 3) Vt=(3, 20, 20)
U=(1,) D=(1,) Vt=(10, 93, 173)
U=(1,) D=(1,) Vt=(33, 356, 1691)
U=(1,) D=(1,) Vt=(113, 877, 11385)
U=(1,) D=(1,) Vt=(267, 376, 11385)
U=(1,) D=(1,) V

## Results

Ground State Energy = -9.7662168 (t=1, U=1)

Total Time          = 50 seconds

## ITensor

In [5]:
using ITensors
@time begin
let
  N = 10
  Npart = 10
  t1 = 1.0
  t2 = 0.0
  U  = 1.0
  V1 = 0.0

  sites = siteinds("Electron",N; conserve_qns=true)

  ampo = AutoMPO()
  for i=1:N
    ampo += (U,"Nupdn",i)
  end
  for b=1:N-1
    ampo += (-t1,"Cdagup",b,"Cup",b+1)
    ampo += (-t1,"Cdagup",b+1,"Cup",b)
    ampo += (-t1,"Cdagdn",b,"Cdn",b+1)
    ampo += (-t1,"Cdagdn",b+1,"Cdn",b)
    ampo += (V1,"Ntot",b,"Ntot",b+1)
  end
  for b=1:N-2
    ampo += (-t2,"Cdagup",b,"Cup",b+2)
    ampo += (-t2,"Cdagup",b+2,"Cup",b)
    ampo += (-t2,"Cdagdn",b,"Cdn",b+2)
    ampo += (-t2,"Cdagdn",b+2,"Cdn",b)
  end
  H = MPO(ampo,sites)

  sweeps = Sweeps(6)
  maxdim!(sweeps,50,100,200,400,800,800)
  cutoff!(sweeps,1E-12)
  @show sweeps

  state = ["Emp" for n=1:N]
  p = Npart
  for i=N:-1:1
    if p > i
      println("Doubly occupying site $i")
      state[i] = "UpDn"
      p -= 2
    elseif p > 0
      println("Singly occupying site $i")
      state[i] = (isodd(i) ? "Up" : "Dn")
      p -= 1
    end
  end
  # Initialize wavefunction to be bond 
  # dimension 10 random MPS with number
  # of particles the same as `state`
  psi0 = randomMPS(sites,state,10)

  # Check total number of particles:
  @show flux(psi0)

  # Start DMRG calculation:
  energy,psi = dmrg(H,psi0,sweeps)

  upd = fill(0.0,N)
  dnd = fill(0.0,N)
  for j=1:N
    orthogonalize!(psi,j)
    psidag_j = dag(prime(psi[j], "Site"))
    upd[j] = scalar(psidag_j * op(sites, "Nup", j) * psi[j])
    dnd[j] = scalar(psidag_j * op(sites, "Ndn", j) * psi[j])
  end

  println("Up Density:")
  for j=1:N
    println("$j $(upd[j])")
  end
  println()

  println("Dn Density:")
  for j=1:N
    println("$j $(dnd[j])")
  end
  println()

  println("Total Density:")
  for j=1:N
    println("$j $(upd[j]+dnd[j])")
  end
  println()

  println("\nGround State Energy = $energy")

end
end

LoadError: MethodError: no method matching MPO(::OpSum, ::Vector{Index{Vector{Pair{QN, Int64}}}})
[0mClosest candidates are:
[0m  MPO(::Any, ::Any, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Int64[39m, [91m::String[39m, [91m::Any[39m, [91m::Any[39m) at /home/vijay/Documents/codes/julia/spin_adapted_operators/src/mpo.jl:17
[0m  MPO(::Any, ::Any, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m) at /home/vijay/Documents/codes/julia/spin_adapted_operators/src/mpo.jl:17

## Results (ITensor)

Ground State Energy = -9.7662747 (t=1, U=1)

Total Time          = 80 seconds